## パラメータの最適化

### 前置き

* 5章では効率的な微分の計算方法（誤差逆伝播法）を学んだ
* 6.1では効率的なパラメータ更新の更新方法を学ぶ

#### 参考

ぶっちゃけこれらの記事の方が分かりやすく、網羅的です
* [【決定版】スーパーわかりやすい最適化アルゴリズム -損失関数からAdamとニュートン法-](https://qiita.com/omiita/items/1735c1d048fe5f611f80)
* [深層学習の最適化アルゴリズム](https://qiita.com/ZoneTsuyoshi/items/8ef6fa1e154d176e25b8)

### 確率的勾配降下法（SGD）
[ソースはこちら](../../../common/presentation/optimizer.py)
* 各地点で最も傾きが急な方向に進む
* ミニバッチ学習の場合は確率的勾配降下法、全データの学習の場合は最急降下法（あるいは単なる勾配降下法）と呼ばれる
* 勾配の方向と最小値の方向が一致しない場合非効率な探索になる

![SGD](fig/SGD.png)

### Momentum
[ソースはこちら](../../../common/presentation/optimizer.py)
* モーメンタムとは運動量のこと
* 球体を斜面に置いた時、勾配方向にチカラを受けて加速していく物理法則を取り入れたもの
* 直前の地点の速度も加えることで慣性(Momentum)をもたせている
* SGDより振動しにくい
* パラメータの推奨値は
    * $\alpha=0.9$
    * $\eta=0.01$
![momentum](fig/momentum.png)

モーメンタムのイメージ図<br>
引用：https://qiita.com/omiita/items/1735c1d048fe5f611f80#53-%E3%83%A2%E3%83%BC%E3%83%A1%E3%83%B3%E3%82%BF%E3%83%A0%E3%82%92%E5%9B%B3%E3%81%A7%E7%90%86%E8%A7%A3
<br>
![モーメンタム図解](fig/モーメンタム図解.png)

### Nesterovの加速法（Nesterov's Accelerated Gradient method)
[ソースはこちら](../../../common/presentation/optimizer.py)

* Momentumをさらに収束への加速を増したもの

詳しくは↓こういうことらしい(https://qiita.com/ophhdn/items/3deaf4fcc43b4f027458)

![ネステロフ](fig/ネステロフの説明.png)

上記の説明とサンプルソース（の数式）が一致するのか確認できていないので勉強会では実装部分は触れません。<br>
ちなみにサンプルソースは↓こんな感じです

$$ {w_{n-1}}' = w_{n-1} + m^2 * v_{n-1}$$

$$ {w_{n}} = {w_{n-1}}' - (1+m) * {\rm{lr}} * \frac{\partial L}{\partial w_n} $$

$$ {v_{n-1}}' = v_{n-1} * m $$

$$ v_{n} = {v_{n-1}}' - {\rm{lr}} * \frac{\partial L}{\partial w_n} $$

$$

### AdaGrad
[ソースはこちら](../../../common/presentation/optimizer.py)
* パラメータ空間の**次元ごと**に適応的に学習係数を調整しながら学習を進める方法
* Momentumでは振動抑制、NAGでは収束速度を改善したが、AdaGradでは **収束方向の情報(=学習度合)** を取り入れた
* 始めは学習係数を大きくして1ステップあたりの更新量を大きくとり、徐々に学習係数を小さくして慎重に学習を進めるようにする
    * 学習が進んだ次元(=傾きが急な)方向への更新度合を小さくしていくことで、学習が遅い次元(=傾きが緩やかな)方向への学習を促す
* 具体的には以下のように実現する：
    - 過去の勾配を2乗和で保持し続ける（2乗和なので単調増加し続ける）
    - 過去の勾配を2乗和の平方根で学習率を割る（これにより徐々に学習スケールが小さくなる）
* パラメータの推奨値は
    * $\eta=0.001（サンプルは0.01）$

![AdaGrad](fig/AdaGrad.png)

### RMSprop
[ソースはこちら](../../../common/presentation/optimizer.py)

* AdaGradの改良版
* AdaGradは過去の更新をすべて保持するため、多段的な傾斜に対応できなかった（減少した更新量を回復する機能をもたないため）
* RMSpropでは過去の更新量が指数的に減衰するため昔の更新量の影響を受けにくくなっている
* 名前はRoot Mean Square propagation（直訳で2乗平均の平方根の伝搬？）から
* パラメータの推奨値は
    * $\epsilon=10^{-6}, 10^{-8}$
    * ${\rm{dr}} = 0.99$
    * ${\rm{lr}} = 0.01$

![RMSprop](fig/RMSprop.png)

RMSpropの数式（原本　非表示で大丈夫です）

* ${\rm{dr}}$ : decay_rate（減衰率）
* ${\rm{lr}}$ : learning_rate（学習係数）

$$ h_n = {\rm{dr}} * h_{n-1} + (1 - {\rm{dr}}) * (\frac{\partial L}{\partial w})^2 $$

$$ W_n = W_{n-1} - \frac{{\rm{lr}}}{\sqrt{h_n} + \epsilon} * \frac{\partial L}{\partial w}$$

### Adam
[ソースはこちら](../../../common/presentation/optimizer.py)
* MomentumとAdaGradを融合したような手法（どちらかというとMomentumとRMSpropの融合かも）
    * ただしオリジナルに比べて指数移動平均を使用しているため、より直近の値の影響が大きくなるようになっている。
* パラメータの推奨値は
    * $\beta1_1=0.9$
    * $\beta_2=0.999$
    * $\eta=0.001$
    * $\epsilon=10^{−8}$

![Adam](fig/Adam.png)

Adamの数式説明（原本　非表示で大丈夫です）

* 学習率
$$ {\rm{lr}}_n = {\rm{lr}}_{n-1} * \frac{\sqrt{1-{\beta}_2^i}}{1 - {\beta}_1^i} $$

* Momentum
$$ m_n = m_{n-1} + (1 - \beta_1)  * ( \frac{\partial L}{\partial w} - m_{n-1} )$$
$$ \ \ \ \   = \beta_1 * m_{n-1} + (1 - \beta_1)  *  \frac{\partial L}{\partial w} $$

* 学習率調整（AdaGrad）
$$ v_n = v_{n-1} + (1 - \beta_2)  * ((\frac{\partial L}{\partial w} )^2 - v_{n-1} )$$
$$ \ \ \ \   = \beta_2 * v_{n-1} + (1 - \beta_2)  *  (\frac{\partial L}{\partial w})^2 $$

* パラメータ更新
$$ w_n = w_{n-1} - {\rm{lr}}_n * \frac{1}{\sqrt{v_n} + \epsilon} * m_n$$

#### Adamのバイアス補正

Adamではパラメータ更新時のバイアス補正効果がある。<br>
参考：https://ichi.pro/adam-dhi-pura-ningu-no-saitekika-niokeru-saishin-no-torendo-118651198977884

![Adamのバイアス補正](fig/Adamのバイアス補正1.jpg)

![Adamのバイアス補正](fig/Adamのバイアス補正2.jpg)

![Adamのバイアス補正](fig/Adamのバイアス補正3.jpg)

### 結局どれがいいのか？

* すべての問題に対して優れた手法はない
* 問題とハイパーパラメータの設定によって優劣が変わる
* SGDよりほかの手法の方が結果が安定して収束する（らしい）

## MNISTにおけるoptimizerの比較

[ソースはこちら](./optimizer_compare_mnist.py)

### モジュールの概説

#### optimizer_compare_mnist.pyの概略図

![overview_optimizer_compare_mnist.png](fig/overview_optimizer_compare_mnist.png)

#### gradientの概略図

![overview_gradient.png](fig/overview_gradient.png)

#### updateの概略図

![overview_update.png](fig/overview_update.png)